In [40]:
from Crypto.Util.number import *
from Crypto.Util.Padding import *

def rotl(x, y):
    x &= 0xFFFFFFFFFFFFFFFF
    return ((x << y) | (x >> (64 - y))) & 0xFFFFFFFFFFFFFFFF

class MyCipher:
    def __init__(self, s0, s1):
        self.X = s0
        self.Y = s1
        self.mod = 0xFFFFFFFFFFFFFFFF
        self.BLOCK_SIZE = 8
    def get_key_stream(self):
        s0 = self.X
        s1 = self.Y
        sum = (s0 + s1) & self.mod
        s1 ^= s0
        key = []
        for _ in range(8):
            key.append(sum & 0xFF)
            sum >>= 8
        
        self.X = (rotl(s0, 24) ^ s1 ^ (s1 << 16)) & self.mod
        self.Y = rotl(s1, 37) & self.mod
        return key    
    def encrypt(self, pt: bytes):
        ct = b''
        for i in range(0, len(pt), self.BLOCK_SIZE):
            key = self.get_key_stream()
            block = pt[i:i+self.BLOCK_SIZE]
            ct += bytes([block[j] ^ key[j] for j in range(len(block))])
        return ct

In [5]:
ct = b'"G:F\xfe\x8f\xb0<O\xc0\x91\xc8\xa6\x96\xc5\xf7N\xc7n\xaf8\x1c,\xcb\xebY<z\xd7\xd8\xc0-\x08\x8d\xe9\x9e\xd8\xa51\xa8\xfbp\x8f\xd4\x13\xf5m\x8f\x02\xa3\xa9\x9e\xb7\xbb\xaf\xbd\xb9\xdf&Y3\xf3\x80\xb8'

In [6]:
b1 = []
b2 = []
for i in range(4):
    b1.append(ct[8*(2*i):8*(2*i+1)])
    b2.append(ct[8*(2*i+1):8*(2*i+2)])

In [7]:
b1

[b'"G:F\xfe\x8f\xb0<',
 b'N\xc7n\xaf8\x1c,\xcb',
 b'\x08\x8d\xe9\x9e\xd8\xa51\xa8',
 b'\x02\xa3\xa9\x9e\xb7\xbb\xaf\xbd']

In [8]:
b2

[b'O\xc0\x91\xc8\xa6\x96\xc5\xf7',
 b'\xebY<z\xd7\xd8\xc0-',
 b'\xfbp\x8f\xd4\x13\xf5m\x8f',
 b'\xb9\xdf&Y3\xf3\x80\xb8']

In [9]:
i0 = bytes_to_long(b1[0])

In [10]:
i1 = bytes_to_long(b1[1])

In [11]:
t1=rotl(i0,24)

In [20]:
from z3 import *

In [21]:
x0 = BitVec("x0",64)
x1 = BitVec("x1",64)

In [24]:
s = Solver()

In [26]:
s.add(i1==(t1^x1^(x1<<16))&0xFFFFFFFFFFFFFFFF )

In [27]:
s.check()

sat

In [28]:
s.model()

[x1 = 9721458296536132593]

In [29]:
l1 = 9721458296536132593

In [30]:
ac = i0^l1

In [31]:
ac

11866620626942876621

In [41]:
from Crypto.Util.number import *
from Crypto.Util.Padding import *
import os
cipher = MyCipher(i0, ac)

In [42]:
p1 =b""

In [43]:
for i in b2:
    p1+=cipher.encrypt(i)

In [44]:
p1

b'FLAG{x013_ro74te_5hif7!!}\x07\x07\x07\x07\x07\x07\x07'